In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

### 基于双向lstm 预测后一个单词

In [2]:
n_hidden = 5
sentence = (
    'Lorem ipsum dolor sit amet consectetur adipisicing elit '
    'sed do eiusmod tempor incididunt ut labore et dolore magna '
    'aliqua Ut enim ad minim veniam quis nostrud exercitation'
)
word_dict= {w:i for i,w in enumerate(list(set(sentence.split())))}
number_dict = {i:w for i,w in enumerate(list(set(sentence.split())))}
n_class = len(word_dict) # 27个不重复的单词
max_len = len(sentence.split()) # 共27个单词

In [18]:
def make_batch(sentence):
    input_batch = []
    target_batch = []
    words = sentence.split()
    for i, word in enumerate(words[:-1]):
        input = [word_dict[n] for n in words[:(i+1)]]
        input = input + [0] * (max_len - len(input))
        target = word_dict[words[i+1]]
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)
        
    return input_batch, target_batch

In [19]:
input_batch, target_batch = make_batch(sentence)
input_batch = torch.FloatTensor(input_batch) # 26, 27, 27 
# 26 指的是 batch_size, 第一个27指的是 序列长度， 第二个27指得是 独热编码长度
target_batch = torch.LongTensor(target_batch)

In [20]:
class BiLSTM(nn.Module):
    def __init__(self):
        super(BiLSTM, self).__init__()
        
        self.lstm = nn.LSTM(input_size=n_class, 
                           hidden_size=n_hidden,
                           bidirectional=True) # 单双向lstm的区别在此
        self.W = nn.Linear(n_hidden*2, n_class, bias=True)  # 隐藏层 维度 * 2
        self.b = nn.Parameter(torch.ones([n_class]))
        
    def forward(self, X):
        input = X.transpose(0,1) # input-> n_step, batch_size, n_class
        hidden_state = torch.zeros(1*2, len(X), n_hidden) # 隐藏层 维度 * 2
        cell_state = torch.zeros(1*2, len(X), n_hidden)  # (2, 26, 5)
        
        outputs, (_, _) = self.lstm(input, (hidden_state, cell_state)) #   n_step,  batch_size,  n_hidden*2
        outputs = outputs[-1]
        model = self.W(outputs) + self.b # model -> batch_size, n_class 26,27
        return model

In [23]:
model = BiLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [25]:
for epoch in range(10000):
    optimizer.zero_grad()
    output = model(input_batch)
    loss = criterion(output, target_batch)
    if (epoch+1) %1000 == 0:
        print(epoch+1, loss)
    loss.backward()
    optimizer.step()

1000 tensor(1.9501, grad_fn=<NllLossBackward>)
2000 tensor(1.7147, grad_fn=<NllLossBackward>)
3000 tensor(1.6391, grad_fn=<NllLossBackward>)
4000 tensor(1.5588, grad_fn=<NllLossBackward>)
5000 tensor(1.5253, grad_fn=<NllLossBackward>)
6000 tensor(1.5087, grad_fn=<NllLossBackward>)
7000 tensor(1.4973, grad_fn=<NllLossBackward>)
8000 tensor(1.1733, grad_fn=<NllLossBackward>)
9000 tensor(1.0703, grad_fn=<NllLossBackward>)
10000 tensor(0.8568, grad_fn=<NllLossBackward>)


In [26]:
predict = model(input_batch).data.max(1, keepdim=True)[1]
print(sentence)
print([number_dict[n.item()] for n in predict.squeeze()])

Lorem ipsum dolor sit amet consectetur adipisicing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua Ut enim ad minim veniam quis nostrud exercitation
['consectetur', 'consectetur', 'consectetur', 'consectetur', 'consectetur', 'consectetur', 'adipisicing', 'incididunt', 'do', 'eiusmod', 'tempor', 'incididunt', 'ut', 'labore', 'et', 'dolore', 'magna', 'aliqua', 'Ut', 'enim', 'ad', 'minim', 'veniam', 'quis', 'nostrud', 'exercitation']
